In [1]:
import json
import datetime
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import openpyxl
import math
today_foloder="20180405" # str(datetime.datetime.now().year)+str(datetime.datetime.now().month).zfill(2)+str(datetime.datetime.now().day).zfill(2)
sales_file_date="2018-03-31"
date=datetime.datetime.strptime('2018-02-24',"%Y-%m-%d").date()
# date1="2018-03-31" # Week ending Saturday
# date2="2018-02-17" # Week ending Saturday, Not available for 5 days prediction
# https://openweathermap.org/weather-conditions reference of weather group

json_list_date=glob.glob("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/*.json")

In [4]:
xxx=json.load(open(json_list_date[24],"r"))

In [7]:
xxx['43232']

{'base': 'stations',
 'clouds': {'all': 75},
 'cod': 200,
 'coord': {'lat': 39.92, 'lon': -82.87},
 'dt': 1519066560,
 'id': 0,
 'main': {'humidity': 88,
  'pressure': 1018,
  'temp': 289.7,
  'temp_max': 291.15,
  'temp_min': 288.15},
 'name': 'Columbus',
 'sys': {'country': 'US',
  'id': 2191,
  'message': 0.0048,
  'sunrise': 1519042671,
  'sunset': 1519081987,
  'type': 1},
 'visibility': 16093,
 'weather': [{'description': 'moderate rain',
   'icon': '10d',
   'id': 501,
   'main': 'Rain'}],
 'wind': {'deg': 190, 'speed': 5.7}}

In [205]:
sales=pd.read_excel("/home/jian/BiglotsCode/outputs/Output_"+sales_file_date+"/wide_sales_date"+sales_file_date+".xlsx",sheetname="sales")

In [206]:
store_zip=pd.read_excel("/home/jian/Projects/Big_Lots/Other Input/all_store_DMA.xlsx",dtype=str)
store_zip=store_zip[store_zip['zip_cd']!='nan']
store_zip=store_zip[['location_id','zip_cd']]
store_zip=store_zip.drop_duplicates().reset_index()
del store_zip['index']
store_zip['location_id']=store_zip['location_id'].astype(int)

In [207]:
sales_Feb24=sales[['location_id',str(date),str(date+datetime.timedelta(days=-364))]]
sales_Feb24=sales_Feb24[(sales_Feb24[str(date)]>0) & (sales_Feb24[str(date+datetime.timedelta(days=-364))]>0)]
sales_Feb24.reset_index(inplace=True)
del sales_Feb24['index']
sales_Feb24.columns=[['location_id','Sales_2018','Sales_2017']]
sales_Feb24['YoY_Dollar']=sales_Feb24['Sales_2018']-sales_Feb24['Sales_2017']
sales_Feb24['YoY_Pctg']=sales_Feb24['YoY_Dollar']/sales_Feb24['Sales_2017']
sales_Feb24=pd.merge(sales_Feb24,store_zip,on="location_id",how="left")

In [208]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Weather/Weather_Variance/Feb_24_Weather.xlsx",engine="xlsxwriter")

In [209]:
plt.hist(sales_Feb24['YoY_Dollar'],bins=np.arange(math.floor(min(sales_Feb24['YoY_Dollar'])/10000)*10000, (math.floor(max(sales_Feb24['YoY_Dollar'])/10000)+1)*10000, 10000))
plt.title("Gaussian Histogram for Sales in Dollar on Feb 24 2018")
plt.xlabel("YoY_Dollar")
plt.ylabel("Frequency")

fig_YoY_Dollar = plt.gcf()
plt.close()

plt.hist(sales_Feb24['YoY_Pctg'],bins=np.arange(math.floor(min(sales_Feb24['YoY_Pctg'])*10)/10, (math.floor(max(sales_Feb24['YoY_Pctg'])*10)+1)/10, 0.1))
plt.title("Gaussian Histogram for Sales in Percetage on Feb 24 2018")
plt.xlabel("YoY_Pctg")
plt.ylabel("Frequency")

fig_YoY_Pctg = plt.gcf()
plt.close()

In [210]:
Sales_Df_Negtive=sales_Feb24[(sales_Feb24['YoY_Dollar']>=-60000) & (sales_Feb24['YoY_Dollar']<-20000)]
Sales_Df_Negtive.reset_index(inplace=True)
del Sales_Df_Negtive['index']
Sales_Df_Negtive.shape

(256, 6)

In [211]:
Three_weekdays_list=sorted(['2018-02-24','2018-02-23','2018-02-18','2018-02-17','2018-02-16','2018-02-11','2018-02-10','2018-02-09','2018-02-04'])

In [212]:
for date1 in Three_weekdays_list:
    response=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+
                                         str(datetime.datetime.strptime(date1,"%Y-%m-%d").date()+datetime.timedelta(days=-1))
                                         +".json",'r'))
    all_weather_groups=[]
    all_weather_desc=[]
    all_weather_id=[]

    for zip_cd in list(response.keys()):
        weather_group = []
        weather_desc = []
        weather_id = []
        for i in range(len(response[zip_cd]['weather'])):
            weather_group=list(set(weather_group+[response[zip_cd]['weather'][i]['main']]))
            all_weather_groups=list(set(all_weather_groups+weather_group))

            weather_desc=list(set(weather_desc+[response[zip_cd]['weather'][i]['description']]))
            all_weather_desc=list(set(all_weather_desc+weather_desc))

            weather_id=list(set(weather_id+[response[zip_cd]['weather'][i]['id']]))
            all_weather_id=list(set(all_weather_id+weather_id))

In [213]:
for date1 in Three_weekdays_list:
    response=json.load(open("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/api_response_"+today_foloder+"/"+
                                         str(datetime.datetime.strptime(date1,"%Y-%m-%d").date()+datetime.timedelta(days=-1))
                                         +".json",'r'))
    df_weather_agg=pd.DataFrame()
    for zip_cd in list(response.keys()):
        temp_max=response[zip_cd]['main']['temp_max'] * 9/5 - 459.67    
        humidity=response[zip_cd]['main']['humidity']/100    
        pressure=response[zip_cd]['main']['pressure']
        wind_speed=response[zip_cd]['wind']['speed']
        if 'deg' in response[zip_cd]['wind'].keys():
            wind_deg=response[zip_cd]['wind']['deg']
        else:
            wind_deg=np.nan
        all_weather_groups_date_list=[weather_group+'_'+date1 for weather_group in all_weather_groups]

        df_weather_binary=pd.DataFrame(columns=all_weather_groups_date_list,index=[0])
        df_weather_binary=df_weather_binary.fillna(0)
        for k in range(len(response[zip_cd]['weather'])):
            for weather_group in all_weather_groups:
                if response[zip_cd]['weather'][k]['main']==weather_group:
                    df_weather_binary[weather_group+'_'+date1]=1
        time=str(datetime.datetime.fromtimestamp(response[zip_cd]['dt']).time())
        date=str(datetime.datetime.fromtimestamp(response[zip_cd]['dt']).date())
        weather = []
        weather_types=len(response[zip_cd]['weather'])

        for j in range(weather_types):
            weather = weather+[str(response[zip_cd]['weather'][j]['main'])]

        country=response[zip_cd]['sys']['country']
        name=response[zip_cd]['name']

        if 'clouds' in response[zip_cd].keys():
            clouds=response[zip_cd]['clouds']['all']/100
        else: 
            clouds= float('NaN')

        df_weather=pd.DataFrame([{'date'+'_'+date1:date,'time'+'_'+date1:time,'zip_cd':zip_cd,'Cloudiness'+'_'+date1:clouds,'humidity'+'_'+date1:humidity,'pressure'+'_'+date1:pressure,
                                  'temp_max'+'_'+date1:temp_max,'wind_speed'+'_'+date1:wind_speed}])
        df_weather=df_weather.join(df_weather_binary)
        df_weather_agg=df_weather_agg.append(df_weather,ignore_index=False)

    Sales_Df_Negtive=pd.merge(Sales_Df_Negtive,df_weather_agg,on="zip_cd",how="left")

In [214]:
Sales_Df_Negtive.to_excel(writer,"data",index=False)

In [215]:
Sales_Df_Negtive_Columns=Sales_Df_Negtive.columns.tolist()
Clear_df_Columns=["location_id","YoY_Dollar","YoY_Pctg"]+[col for col in Sales_Df_Negtive_Columns if 'Clear' in col]
Rain_df_Columns=["location_id","YoY_Dollar","YoY_Pctg"]+[col for col in Sales_Df_Negtive_Columns if 'Rain' in col]
Snow_df_Columns=["location_id","YoY_Dollar","YoY_Pctg"]+[col for col in Sales_Df_Negtive_Columns if 'Snow' in col]
Temp_df_Columns=["location_id","YoY_Dollar","YoY_Pctg"]+[col for col in Sales_Df_Negtive_Columns if 'temp' in col]

In [216]:
Clear_df=Sales_Df_Negtive[Clear_df_Columns]
Rain_df=Sales_Df_Negtive[Rain_df_Columns]
Snow_df=Sales_Df_Negtive[Snow_df_Columns]
Temp_df=Sales_Df_Negtive[Temp_df_Columns]

In [217]:
def week_day(x):
    y=np.nan
    weekday_num=datetime.datetime.strptime(x,"%Y-%m-%d").date().weekday()
    if weekday_num==0:
        y="Monday"
    if weekday_num==1:
        y="Tuesday"
    if weekday_num==2:
        y="Wednesday"
    if weekday_num==3:
        y="Thursday"
    if weekday_num==4:
        y="Friday"
    if weekday_num==5:
        y="Saturday"
    if weekday_num==6:
        y="Sunday"
    return y

In [218]:
def weather_binary_variance(df,weather):
    
    variance_columns=["Variance_"+week_day(x[(len(x)-10):len(x)])+"_"+x[(len(x)-10):len(x)] for x in df.columns[6:len(df.columns)]]
    for col in Rain_df_Variance_Columns:
        date=col[(len(col)-10):len(col)]
        date_pri=str(datetime.datetime.strptime(date,"%Y-%m-%d").date()+datetime.timedelta(days=-7))
        df[col]=df[weather+"_"+date]-df[weather+"_"+date_pri]
    return df

In [219]:
Clear_df_variance=weather_binary_variance(Clear_df,"Clear")
Rain_df_variance=weather_binary_variance(Rain_df,"Rain")
Snow_df_variance=weather_binary_variance(Snow_df,"Snow")
Temp_df_variance=weather_binary_variance(Temp_df,"temp_max")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [220]:
Temp_df_variance['week1_Sat-Fri']=Temp_df_variance['temp_max_2018-02-10']-Temp_df_variance['temp_max_2018-02-09']
Temp_df_variance['week2_Sat-Fri']=Temp_df_variance['temp_max_2018-02-17']-Temp_df_variance['temp_max_2018-02-16']
Temp_df_variance['week3_Sat-Fri']=Temp_df_variance['temp_max_2018-02-24']-Temp_df_variance['temp_max_2018-02-23']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [221]:
Clear_df_variance.to_excel(writer,"clear",index=False)
Rain_df_variance.to_excel(writer,"rain",index=False)
Snow_df_variance.to_excel(writer,"snow",index=False)
Temp_df_variance.to_excel(writer,"temp_max",index=False)
writer.save()

In [1]:
import pandas as pd
df=pd.DataFrame({"A":[1,2],"B":[3,4]})

In [2]:
df

,A,B
0,1,3
1,2,4


In [3]:
df['C']=0
df['C'][df['B']==3]=1

In [4]:
df

,A,B,C
0,1,3,1
1,2,4,0
